In [2]:
import requests
import ee
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from io import BytesIO

#ee.Authenticate()
ee.Initialize()


In [5]:
#Import squares that contain beaver dams in, pull in images for those square and make into a list

Contains_WY_beaver_dams = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_Wy_Contains').getInfo()['features']

def get_image(feature):
    min_lon = feature['geometry']['coordinates'][0][0][0]
    min_lat = feature['geometry']['coordinates'][0][0][1]
    max_lon = feature['geometry']['coordinates'][0][1][0]
    max_lat = feature['geometry']['coordinates'][0][2][1]
    response = requests.get(
        url='https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/[{},{},{},{}]/250x250?access_token=pk.eyJ1IjoicHVpd2FoIiwiYSI6ImNrbHM1djlhdDBiM28yb28xbmExM2d0M20ifQ.uFj_qHx0c3OTNu5o-Njf4g'.format(min_lon, min_lat, max_lon, max_lat),
            )
    j = Image.open(BytesIO(response.content))
    return j

image_collection_contains = [get_image(x) for x in tqdm(Contains_WY_beaver_dams)]

100%|██████████| 676/676 [03:17<00:00,  3.42it/s]


In [4]:
#Do the same, but for squares that are missing beaver dams

Absent_WY_beaver_dams = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_Wy_Not_Contains').getInfo()['features']

def get_image(feature):
    min_lon = feature['geometry']['coordinates'][0][0][0]
    min_lat = feature['geometry']['coordinates'][0][0][1]
    max_lon = feature['geometry']['coordinates'][0][1][0]
    max_lat = feature['geometry']['coordinates'][0][2][1]
    response = requests.get(
        url='https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/[{},{},{},{}]/250x250?access_token=pk.eyJ1IjoicHVpd2FoIiwiYSI6ImNrbHM1djlhdDBiM28yb28xbmExM2d0M20ifQ.uFj_qHx0c3OTNu5o-Njf4g'.format(min_lon, min_lat, max_lon, max_lat),
            )
    j = Image.open(BytesIO(response.content))
    return j

image_collection_absent = [get_image(x) for x in tqdm(Absent_WY_beaver_dams)]

100%|██████████| 4538/4538 [57:55<00:00,  1.31it/s]  


In [7]:
#label images as having a beaver dam

image_contains_df = pd.DataFrame(image_collection_contains, [1]*len(image_collection_contains))

In [12]:
#reseting index, naming columns

image_contains_df.reset_index(inplace = True)
image_contains_df.rename(columns = {'index': 'Present', 0: 'Image'}, inplace = True)
image_contains_df.head()

,Present,Image
0,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [9]:
#same process for images missing beaver dams

image_absent_df = pd.DataFrame(image_collection_absent, [0]*len(image_collection_absent))

In [13]:
image_absent_df.reset_index(inplace = True)
image_absent_df.rename(columns = {'index': 'Present', 0: 'Image'}, inplace = True)
image_absent_df.head()

,Present,Image
0,0,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,0,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,0,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,0,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,0,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [14]:
#combine into one dataframe

image_df = pd.concat([image_contains_df, image_absent_df])

In [15]:
image_df.head()

,Present,Image
0,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,1,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [16]:
print(len(image_df))

5214


In [26]:
image_df.to_hdf('beaver_images.h5', key = 'image_df')

/Users/amgadellaboudy/.local/lib/python3.8/site-packages/pandas/core/generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Image'], dtype='object')]

  pytables.to_hdf(
